<a href="https://colab.research.google.com/github/Lalith2005reddy/HDR_Image_Segmentation/blob/main/Coco_to_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from pycocotools import mask as maskUtils
from pycocotools.coco import COCO
import matplotlib.pyplot as plt

# Paths to the COCO annotation file and image directory
coco_json_path = r'C:\Users\91630\Desktop\_annotations.coco.json'
image_directory = r'C:\Users\91630\Desktop\ML'  # Correct directory containing images
output_directory = r'C:\Users\91630\Desktop\ML'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Load the COCO annotation file
coco = COCO(coco_json_path)

# List files in the image directory
print("Files in directory:", os.listdir(image_directory))

# Iterate over each image in the COCO dataset
for img_id in coco.getImgIds():
    # Load the image metadata
    img_info = coco.loadImgs(img_id)[0]
    img_filename = img_info['file_name']
    img_path = os.path.join(image_directory, img_filename)

    # Debug: print the image path
    print(f'Processing image: {img_path}')

    # Read the image
    image = cv2.imread(img_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f'Warning: Unable to load image at path {img_path}')
        continue

    height, width, _ = image.shape

    # Create an empty mask
    mask = np.zeros((height, width), dtype=np.uint8)

    # Load annotations for the image
    ann_ids = coco.getAnnIds(imgIds=img_info['id'])
    anns = coco.loadAnns(ann_ids)

    # Iterate over each annotation and create the mask
    for ann in anns:
        # Convert the annotation to a binary mask
        rle = maskUtils.frPyObjects(ann['segmentation'], height, width)
        m = maskUtils.decode(rle)

        # Squeeze the mask to remove the extra dimension
        m = np.squeeze(m)

        # Add the mask to the output mask
        mask = np.maximum(mask, m * ann['category_id'])  # Or use a fixed value (e.g., 255) for the mask

    # Save the mask as an image
    mask_output_path = os.path.join(output_directory, f"{os.path.splitext(img_filename)[0]}_mask.png")
    cv2.imwrite(mask_output_path, mask)
    print(f'Saved mask to {mask_output_path}')

    # Optionally, visualize the mask
    plt.imshow(mask, cmap='gray')
    plt.title(f"Mask for {img_filename}")
    plt.show()
